<a href="https://colab.research.google.com/github/Arif432/portfolios/blob/main/bertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Define a function to process user input
# def process_user_input(user_query):
#     # Tokenize the input
#     tokenized_input = tokenizer.encode_plus(user_query, return_tensors="pt", padding=True, truncation=True)

#     # Get contextualized embeddings
#     with torch.no_grad():
#         output = model(**tokenized_input)
#         embeddings = output.last_hidden_state

#     symptom_embedding = embeddings[0]
#     symptom = tokenizer.decode(tokenized_input["input_ids"][0])

#     return symptom

# # Example usage
# user_query = "Hi can you please help me with diabetes?"
# symptom = process_user_input(user_query)
# print(f"Detected symptom: {symptom}")


Detected symptom: [CLS] hi can you please help me with diabetes? [SEP]


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import torch.nn.functional as F
tokenizer = AutoTokenizer.from_pretrained("anindabitm/sagemaker-BioclinicalBERT-ADR")
model = AutoModelForQuestionAnswering.from_pretrained("anindabitm/sagemaker-BioclinicalBERT-ADR")
input_text = "Patient was diagnosed with diabetes at the age of forty. He has been managing it with insulin injections."
question = "What age was the patient diagnosed with diabetes?"
inputs = tokenizer.encode_plus(question, input_text, add_special_tokens=True, return_tensors="pt")
start_logits, end_logits = model(**inputs).start_logits, model(**inputs).end_logits
span_scores = F.softmax(start_logits, dim=1) * F.softmax(end_logits, dim=1)
start_index, end_index = torch.unravel_index(span_scores.argmax(), span_scores.shape)
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
answer = tokenizer.convert_tokens_to_string(tokens[start_index:end_index + 1])
print("Answer:", answer)


Answer: [CLS] what age was the patient diagnosed with diabetes? [SEP] patient was diagnosed with diabetes


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForQuestionAnswering.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
context = "Patient was diagnosed with diabetes at the age of 40. He has been managing it with insulin injections."
question = "What age was the patient diagnosed with diabetes?"
inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
print("Answer:", answer)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Answer: 


In [ ]:


import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
# Load your question-answer dataset from "diabetes.json"
with open("diabetes.json", "r") as f:
    data = json.load(f)
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("anindabitm/sagemaker-BioclinicalBERT-ADR")
model = AutoModelForQuestionAnswering.from_pretrained("anindabitm/sagemaker-BioclinicalBERT-ADR")
max_length = 512  # or any other suitable length

# Tokenize and preprocess your data with padding
tokenized_data = []
for item in data:
    context = item["answer"]
    question = item["question"]
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length")
    tokenized_data.append({
        "input_ids": inputs["input_ids"].flatten(),
        "attention_mask": inputs["attention_mask"].flatten(),
        "start_positions": torch.tensor(tokenizer.encode(item["answer"])[1], dtype=torch.long),
        "end_positions": torch.tensor(tokenizer.encode(item["answer"])[-1], dtype=torch.long),
    })


# Set training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=3,
    output_dir="./output",
    logging_dir="./logs",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-bert-diabetes-sagemaker-model")


tokenizer_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

Step,Training Loss


In [ ]:
# !pip install accelerate==0.21.0
# !pip install transformers[torch]

In [ ]:
# !pip install transformers accelerate -U


In [ ]:
# testing what is trained

# Load the fine-tuned model
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-bert-diabetes-sagemaker-model")
tokenizer = AutoTokenizer.from_pretrained("anindabitm/sagemaker-BioclinicalBERT-ADR")

# Prepare the test data
test_data = [
    {"question": "why am i always hungry and tired? . i am a 19 year old male 6'0\" tall 150 pounds. i work out at least 4 times a week including cardio and weight training. i eat healthy and balanced with the occasional junk food. i don't drink caffeine or soda. i always eat breakfast lunch and dinner and snack throughout the whole day. some days i eat over 3500 calories. i don't gain or lose weight easily. i sleep between 8-10 hours every night including the weekends. however i am constantly hungry even after a meal and i am always tired.",
     "answer": "there could be a few things going on. even though you eat healthy you could be low in nutrients causing you to feel tired. the only way to know for sure is to go for a physical and get labs run including vitamin d glucose and iron. also less likely increased hunger accompanied with thirst and weight loss could be sign of diabetes. also look closely at the quality of your diet. are you eating filling foods with meals including healthy fats (nuts and seeds/avocado/veg oils) whole grains lean proteins dairy beans/legumes and fruits and veggies? the fiber found in plant foods such as fruits vegetables whole grains and beans help fill you up. fat provides adequate calories and is satisfying and protein is also filling. make sure your meals contain 4-5 food groups and snacks 2-3. you want to avoid grazing on less filling snacks high in refined grains and low in nutrients (crackers pretzels chips). if you need help see a registered dietitian. you can find one in your area at eatright. org. read more. filling foods do you feel hungry and tired on days when you eat more than 3500 calories? probably there is mismatch between calorie intake and expenditure. feeling hungry is ok with your workout schedule but feeling tired is worrisome. any family history of diabetes please check. there are a few reasons you are feeling tired without any reason. one of them is anemia which is a lack of red blood cells and conversely oxygen from the lungs is not properly brought to the tissues and cells. anemia may be caused by deficiency of vitamins or minerals internal bleeding or chronic diseases. women who are at “that time” of the month are susceptible to anemia as blood is lost during menstruation. still iron deficiency due to menstruation is less than in pregnant women or lactating mothers as their body needs extra iron to maintain healthy blood levels. things may also go wrong when your thyroid glands refuse to cooperate with you. thyroid hormones thyroxine (t4) and triiodothyronine (t3) are responsible for metabolic processes in the body. too much thyroid hormone causes hyperthyroidism which speeds up metabolism while too little thyroid hormone slows down metabolism. hyperthyroidism causes fatigue and muscle weakness and starts in the thighs. doing energy demanding activities become difficult and other symptoms include weight loss feeling warm shorter menstruations increased heart rate and thirst. hypothyroidism also causes fatigue and muscle weakness. the symptoms are reversed to hyperthyroidism such as weight gain feeling cold and longer menstruations. diabetes notorious for causing a range of problems in the body is also related to fatigue. as glucose is the staple fuel of the body it is not utilized properly in patients with type 2 diabetes as the absence of insulin causes the glucose to build up in the body. without its staple fuel the tissues of the body are not nourished properly and causes fatigue. sadly being sad can also be the reason you feel tired. depression induces negative feelings and also has negative effects on the body as it causes a reduction in energy levels changes in sleep and eating patterns decreased concentration and overall laziness and worthlessness which keeps you in bed all day"},
    {"question": "can you safely take mobic for arthritis pain if you are taking cymbalta and verapamil?",
     "answer": "mobic can be taken safely with those two medications. there are no significant interactions between the medications you listed. mobic is an anti-inflammatory medication and does have a slight tendency to raise blood pressure when taken regularly. so it is important to watch your blood pressure while you are taking it. i am sure your doctor will be monitoring for that. also to avoid dangerous drug interactions it's always important to have all your prescriptions filled at the same pharmacy so the pharmacist has access to all the information he or she needs to take the best care of you. also new prescriptions will be screened against all the other medications you are taking to ensure there are no problems"},
]

# Tokenize and preprocess the test data
tokenized_test_data = []
for item in test_data:
    context = item["answer"]
    question = item["question"]
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    tokenized_test_data.append(inputs)

# Perform inference
for inputs in tokenized_test_data:
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted answer
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    answer_tokens = inputs["input_ids"][0][start_index : end_index + 1]
    predicted_answer = tokenizer.decode(answer_tokens)

    print("Question:", tokenizer.decode(inputs["input_ids"][0]))
    print("Predicted Answer:", predicted_answer)


Question: [CLS] why am i always hungry and tired?. i am a 19 year old male 6'0 " tall 150 pounds. i work out at least 4 times a week including cardio and weight training. i eat healthy and balanced with the occasional junk food. i don't drink caffeine or soda. i always eat breakfast lunch and dinner and snack throughout the whole day. some days i eat over 3500 calories. i don't gain or lose weight easily. i sleep between 8 - 10 hours every night including the weekends. however i am constantly hungry even after a meal and i am always tired. [SEP] there could be a few things going on. even though you eat healthy you could be low in nutrients causing you to feel tired. the only way to know for sure is to go for a physical and get labs run including vitamin d glucose and iron. also less likely increased hunger accompanied with thirst and weight loss could be sign of diabetes. also look closely at the quality of your diet. are you eating filling foods with meals including healthy fats ( nut